In [1]:
# VGG Like Model With Batchnorm

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
np.random.seed(12345)
#%matplotlib inline

/home/pdeepsingh91/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train_images_path = keras.utils.get_file('train-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz')
train_labels_path = keras.utils.get_file('train-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-labels-idx1-ubyte.gz')
test_images_path = keras.utils.get_file('t10k-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-images-idx3-ubyte.gz')
test_labels_path = keras.utils.get_file('t10k-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-labels-idx1-ubyte.gz')

In [4]:
batch_size = 512

In [5]:
def load_mnist(images_path, labels_path):
    import os
    import gzip
    import numpy as np

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [6]:
X_train_orig, y_train_orig = load_mnist(train_images_path, train_labels_path)
X_test, y_test = load_mnist(test_images_path, test_labels_path)
X_train_orig = X_train_orig.astype('float32')
X_test = X_test.astype('float32')
X_train_orig /= 255
X_test /= 255

In [7]:
print(X_train_orig.shape)
print(y_train_orig.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, test_size=0.2, random_state=12345)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(48000, 784)
(48000,)
(12000, 784)
(12000,)


In [10]:
#plt.imshow(X_train[1, :].reshape((28, 28)))

In [11]:
img_rows = 28
img_cols = 28
input_shape = (img_rows, img_cols, 1)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

In [12]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [13]:
cnn5 = Sequential([
    Lambda(norm_input, input_shape=(28,28, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),

    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),

    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


In [14]:
cnn5.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [15]:
cnn5.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1239s 26ms/step - loss: 0.8209 - acc: 0.7228 - val_loss: 2.4042 - val_acc: 0.3893
Epoch 2/10
48000/48000 [==============================] - 1241s 26ms/step - loss: 0.4500 - acc: 0.8379 - val_loss: 0.6855 - val_acc: 0.7822
Epoch 3/10
48000/48000 [==============================] - 1241s 26ms/step - loss: 0.3601 - acc: 0.8706 - val_loss: 0.5330 - val_acc: 0.8159
Epoch 4/10
48000/48000 [==============================] - 1242s 26ms/step - loss: 0.3085 - acc: 0.8885 - val_loss: 0.3139 - val_acc: 0.8897
Epoch 5/10
48000/48000 [==============================] - 1238s 26ms/step - loss: 0.2797 - acc: 0.8982 - val_loss: 0.2345 - val_acc: 0.9176
Epoch 6/10
48000/48000 [==============================] - 1241s 26ms/step - loss: 0.2598 - acc: 0.9053 - val_loss: 0.2871 - val_acc: 0.9028
Epoch 7/10
48000/48000 [==============================] - 1240s 26ms/step - loss: 0.2377 - acc: 0.9144 - val_l

In [16]:
cnn5.optimizer.lr = 0.0001

In [17]:
cnn5.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1229s 26ms/step - loss: 0.1950 - acc: 0.9274 - val_loss: 0.2249 - val_acc: 0.9236
Epoch 2/10
48000/48000 [==============================] - 1225s 26ms/step - loss: 0.1800 - acc: 0.9341 - val_loss: 0.2204 - val_acc: 0.9233
Epoch 3/10
48000/48000 [==============================] - 1225s 26ms/step - loss: 0.1765 - acc: 0.9351 - val_loss: 0.1828 - val_acc: 0.9349
Epoch 4/10
48000/48000 [==============================] - 1223s 25ms/step - loss: 0.1669 - acc: 0.9385 - val_loss: 0.1882 - val_acc: 0.9359
Epoch 5/10
48000/48000 [==============================] - 1225s 26ms/step - loss: 0.1609 - acc: 0.9412 - val_loss: 0.2244 - val_acc: 0.9244
Epoch 6/10
48000/48000 [==============================] - 1223s 25ms/step - loss: 0.1547 - acc: 0.9434 - val_loss: 0.1994 - val_acc: 0.9343
Epoch 7/10
48000/48000 [==============================] - 1223s 25ms/step - loss: 0.1419 - acc: 0.9480 - val_l

In [20]:
score = cnn5.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.21374387829452754
Test accuracy: 0.9325


In [15]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=batch_size)
val_batches = gen.flow(X_val, y_val, batch_size=batch_size)


In [16]:
cnn5.fit_generator(batches, steps_per_epoch=48000//batch_size, epochs=50,
                    validation_data=val_batches, validation_steps=12000//batch_size, use_multiprocessing=True)

Epoch 1/50
93/93 [==============================] - 1222s 13s/step - loss: 1.0081 - acc: 0.6548 - val_loss: 0.7504 - val_acc: 0.7347
Epoch 2/50
93/93 [==============================] - 1223s 13s/step - loss: 0.6181 - acc: 0.7673 - val_loss: 1.3811 - val_acc: 0.5908
Epoch 3/50
93/93 [==============================] - 1220s 13s/step - loss: 0.5011 - acc: 0.8147 - val_loss: 1.5170 - val_acc: 0.5789
Epoch 4/50
93/93 [==============================] - 1223s 13s/step - loss: 0.4328 - acc: 0.8379 - val_loss: 0.7951 - val_acc: 0.7198
Epoch 5/50
93/93 [==============================] - 1215s 13s/step - loss: 0.3933 - acc: 0.8562 - val_loss: 0.7945 - val_acc: 0.7313
Epoch 6/50
93/93 [==============================] - 1217s 13s/step - loss: 0.3613 - acc: 0.8678 - val_loss: 0.3355 - val_acc: 0.8793
Epoch 7/50
93/93 [==============================] - 1221s 13s/step - loss: 0.3433 - acc: 0.8743 - val_loss: 0.3183 - val_acc: 0.8845
Epoch 8/50
93/93 [==============================] - 1216s 13s/step - 

In [18]:
score = cnn5.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.1863873804986477
Test accuracy: 0.9371
